In [6]:
from sqlalchemy import ForeignKey, select, delete, String, and_
from sqlalchemy.orm import Session
from database import MarkedSentence, LessonSession, engine

In [7]:
stmt = select(LessonSession)
with Session(engine) as session:
    for entry in session.scalars(stmt):
        print(entry)

2024-09-07 13:16:05,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-07 13:16:05,454 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions
2024-09-07 13:16:05,455 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ()
date time : datetime.datetime(2024, 9, 4, 22, 42, 30, 759328), lesson 50 errors number : 6
date time : datetime.datetime(2024, 9, 4, 23, 2, 0, 780033), lesson 1 errors number : 0
date time : datetime.datetime(2024, 9, 5, 9, 19, 11, 922138), lesson 51 errors number : 3
date time : datetime.datetime(2024, 9, 6, 23, 30, 29, 589456), lesson 52 errors number : 5
date time : datetime.datetime(2024, 9, 6, 23, 54, 35, 589923), lesson 3 errors number : 3
2024-09-07 13:16:05,458 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
stmt = select(LessonSession)
with Session(engine) as session:
    for entry in session.scalars(stmt):
        for sentence in entry.errored_sentences:
            print(sentence.session)
            

2024-09-07 13:16:06,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-07 13:16:06,495 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions
2024-09-07 13:16:06,496 INFO sqlalchemy.engine.Engine [cached since 1.042s ago] ()
2024-09-07 13:16:06,522 INFO sqlalchemy.engine.Engine SELECT marked_sentences.id AS marked_sentences_id, marked_sentences.sentence AS marked_sentences_sentence, marked_sentences.comment AS marked_sentences_comment, marked_sentences.line AS marked_sentences_line, marked_sentences.session_id AS marked_sentences_session_id 
FROM marked_sentences 
WHERE ? = marked_sentences.session_id
2024-09-07 13:16:06,522 INFO sqlalchemy.engine.Engine [generated in 0.00072s] (1,)
date time : datetime.datetime(2024, 9, 4, 22, 42, 30, 759328), lesson 50 errors number : 6
date time : datetime.datetime(2024, 9, 4, 22, 42, 30, 759328), lesson 50 errors number : 6
date time : d

In [9]:
from database import get_lesson_sessions_history

In [15]:
sessions = get_lesson_sessions_history()

----------------- begin_date : None
----------------- end_date : None
2024-09-07 13:18:14,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-07 13:18:14,724 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions 
WHERE lesson_sessions.date_time >= ? AND lesson_sessions.date_time <= ? AND lesson_sessions.lesson <= ? AND lesson_sessions.lesson >= ?
2024-09-07 13:18:14,726 INFO sqlalchemy.engine.Engine [cached since 119s ago] ('0001-01-01 00:00:00.000000', '9999-12-31 23:59:59.999999', 100, 1)
2024-09-07 13:18:14,729 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
sessions

{50: {datetime.datetime(2024, 9, 4, 22, 42, 30, 759328): 6},
 1: {datetime.datetime(2024, 9, 4, 23, 2, 0, 780033): 0},
 51: {datetime.datetime(2024, 9, 5, 9, 19, 11, 922138): 3},
 3: {datetime.datetime(2024, 9, 6, 23, 54, 35, 589923): 3}}

In [17]:
import datetime

In [18]:
stmt = select(LessonSession)
with Session(engine) as session:
    session.execute(delete(LessonSession).where(LessonSession.date_time == datetime.datetime(2024, 9, 6, 23, 30, 29, 589456)))
    session.commit()

2024-09-07 13:18:28,314 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-07 13:18:28,316 INFO sqlalchemy.engine.Engine DELETE FROM lesson_sessions WHERE lesson_sessions.date_time = ?
2024-09-07 13:18:28,317 INFO sqlalchemy.engine.Engine [cached since 175.2s ago] ('2024-09-06 23:30:29.589456',)
2024-09-07 13:18:28,318 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
html_with_selection = """<p> N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos </p>                  <p> S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?  </p>                  <p> S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?  </p>                  <p> S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.  </p>                  <p> S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.  </p>                  <p> S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be<b>bí</b>an <b>san</b>gre!  </p>                  <p> S08-¿<b>Pe</b>ro <b>nun</b>ca lo <b>to</b>man <b></b><mark class='V' title='3ème personne du pluriel du présent du verbe tomar.'><b>dul</b>ce</mark>?  </p>                  <p> S09-Sí, <b>cla</b>ro, hoy en <b>dí</b>a tam<b>bién</b> <mark class='G' title='cacao est complément d&#39;objet indirect donc="le" et non pas "lo"'>le</mark> <b>e</b>chan a<b>zú</b>car,  </p>                  <p> S10-<b>pe</b>ro en la tradi<b>ción</b> in<b>dí</b>gena no se <mark class='G' title='cacao est complément d&#39;objet indirect donc="le" et non pas "lo"'>le</mark> a<b>ña</b>de <b>le</b>che, <b>so</b>lo <b>a</b>gua.  </p>                  <p> S11-<b></b><mark class='O'><b></b><mark class='O' title='Meriendas est féminin, donc nuestras et non pas nuestros.'><b>Nues</b>tras</mark></mark> me<b>rien</b>das sí que las e<b>cha</b>ba de <b>me</b>nos un buen choco<b>la</b>te con <b>chu</b>rros. . .  </p>                  <p> S12-<b>E</b>so, <b>pa</b>ra meren<b>dar</b>, ¡<b>na</b>da <b>co</b>mo Es<b>pa</b>ña!  </p>                  <p> T00-TRANSLATE-Ejer<b>ci</b>cio 1 – Tra<b>duz</b>ca </p>                  <p> T01-No me has <b>di</b>cho si te gus<b>tó</b> la <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> T02-¿Te atre<b>vis</b>te a to<b>mar</b> choco<b>la</b>te a<b>mar</b>go, pi<b>can</b>te y sin a<b>zú</b>car?  </p>                  <p> T03-¡Qué va! Me dio <b>as</b>co y no lo pro<b>bé</b>.  </p>                  <p> T04-Pues a lo me<b>jor</b> es<b>tá</b> muy <b>ri</b>co, <b>pe</b>ro pre<b>fie</b>ro un choco<b>la</b>te con <b>chu</b>rros.  </p>                  <p> T05-<b>E</b>cho de <b>me</b>nos la co<b>ci</b>na mexi<b>ca</b>na.  </p>"""              

In [20]:
print(html_with_selection)

<p> N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos </p>                  <p> S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?  </p>                  <p> S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?  </p>                  <p> S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.  </p>                  <p> S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.  </p>                  <p> S07-¡Por <b>e</

In [3]:
html_with_selection

'<p> N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos </p>                  <p> S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na </p>                  <p> S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?  </p>                  <p> S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.  </p>                  <p> S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class=\'V\' title=\'3ème personne du singulier du passé simple de dar : dio et non pas di.\'>dio</mark> <b>as</b>co?  </p>                  <p> S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.  </p>                  <p> S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.  </p>                  <p> S07-¡Por <

In [21]:
from selection import extract_selection

In [23]:
marked_lines_numbers, sentences = extract_selection(html_with_selection)

attr_name : class, attr_value : V
---- in convert_to_html_escape, returning V
---- escaped_attr_value : V
attr_name = class, char : V, 86, 0
attr_name : title, attr_value : 3ème personne du singulier du passé simple de dar : dio et non pas di.
---- in convert_to_html_escape, returning 3ème personne du singulier du passé simple de dar : dio et non pas di.
---- escaped_attr_value : 3ème personne du singulier du passé simple de dar : dio et non pas di.
attr_name = title, char : 3, 51, 0
attr_name = title, char : è, 232, 1
attr_name = title, char : m, 109, 2
attr_name = title, char : e, 101, 3
attr_name = title, char :  , 32, 4
attr_name = title, char : p, 112, 5
attr_name = title, char : e, 101, 6
attr_name = title, char : r, 114, 7
attr_name = title, char : s, 115, 8
attr_name = title, char : o, 111, 9
attr_name = title, char : n, 110, 10
attr_name = title, char : n, 110, 11
attr_name = title, char : e, 101, 12
attr_name = title, char :  , 32, 13
attr_name = title, char : d, 100, 14
attr

In [24]:
marked_lines_numbers

{5: True, 9: True, 10: True, 11: True, 12: True}

In [25]:
sentences

['N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos',
 'S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na',
 'S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na',
 'S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?',
 'S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.',
 "S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?",
 'S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.',
 'S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.',
 'S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be<b>bí</b>an <b>san</b>gre!',
 "S08-¿<b>Pe</b>ro <b>nun</b>ca lo <b>to</b>man <b></b><mark class='V' title='3ème personne du plurie

In [26]:
for index, sentence in enumerate(sentences):
    print(index, sentence)

0 N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos
1 S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na
2 S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na
3 S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?
4 S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.
5 S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?
6 S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.
7 S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.
8 S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be<b>bí</b>an <b>san</b>gre!
9 S08-¿<b>Pe</b>ro <b>nun</b>ca lo <b>to</b>man <b></b><mark class='V' title='3ème personne du pluriel du présent du ve

In [27]:
from selection import store_lesson_errors

In [28]:
store_lesson_errors(52, sentences, marked_lines_numbers, datetime.datetime(2024, 9, 6, 23, 30, 29, 589456))

-------------- In store_lesson_errors
---- 0 ------ N52-Lec<b>ción</b> cin<b>cuen</b>ta y dos
---- 1 ------ S00-TITLE-La co<b>ci</b>na mexi<b>ca</b>na
---- 2 ------ S01-No me has <b>di</b>cho <b>na</b>da de la co<b>ci</b>na mexi<b>ca</b>na
---- 3 ------ S02-¿O a lo me<b>jor</b> no te atre<b>vis</b>te a pro<b>bar</b>la?
---- 4 ------ S03-¡Por su<b>pues</b>to que la pro<b>bé</b>! Me gus<b>tó</b> <b>so</b>bre <b>to</b>do el <b>mo</b>le es <b>u</b>na <b>sal</b>sa de ca<b>ca</b>o y guin<b>di</b>llas.
---- 5 ------ S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?
---- 6 ------ S05-¡Qué va! ¡Es<b>tá</b> muy <b>ri</b>co! Lo a<b>mar</b>go va muy bien con lo pi<b>can</b>te.
---- 7 ------ S06-Los <b>pue</b>blos precolom<b>bi</b>nos ha<b>cí</b>an <b>u</b>na be<b>bi</b>da <b>ro</b>ja con el ca<b>ca</b>o.
---- 8 ------ S07-¡Por <b>e</b>so los conquista<b>do</b>res cre<b>ye</b>ron que be

In [29]:
select

<function sqlalchemy.sql._selectable_constructors.select(*entities: '_ColumnsClauseArgument[Any]', **__kw: 'Any') -> 'Select[Any]'>

In [30]:
from database import MarkedSentence

In [31]:
stmt = select(LessonSession).where(
                        LessonSession.lesson == 52)

with Session(engine) as session:
    for entry in session.scalars(stmt):
        print(entry.errored_sentences)
        sfdb = entry.errored_sentences

2024-09-07 13:25:50,255 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-07 13:25:50,257 INFO sqlalchemy.engine.Engine SELECT lesson_sessions.id, lesson_sessions.lesson, lesson_sessions.date_time, lesson_sessions.errors_number 
FROM lesson_sessions 
WHERE lesson_sessions.lesson = ?
2024-09-07 13:25:50,258 INFO sqlalchemy.engine.Engine [generated in 0.00101s] (52,)
2024-09-07 13:25:50,261 INFO sqlalchemy.engine.Engine SELECT marked_sentences.id AS marked_sentences_id, marked_sentences.sentence AS marked_sentences_sentence, marked_sentences.comment AS marked_sentences_comment, marked_sentences.line AS marked_sentences_line, marked_sentences.session_id AS marked_sentences_session_id 
FROM marked_sentences 
WHERE ? = marked_sentences.session_id
2024-09-07 13:25:50,262 INFO sqlalchemy.engine.Engine [cached since 583.8s ago] (6,)
[Sentence(id=18, line=5, sentence ="S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : di

In [32]:
sfdb

[Sentence(id=18, line=5, sentence ="S04-¿Choco<b>la</b>te pi<b>can</b>te? ¿No te <mark class='V' title='3ème personne du singulier du passé simple de dar : dio et non pas di.'>dio</mark> <b>as</b>co?", comment=None),
 Sentence(id=19, line=9, sentence ="S08-¿<b>Pe</b>ro <b>nun</b>ca lo <b>to</b>man <b></b><mark class='V' title='3ème personne du pluriel du présent du verbe tomar.'><b>dul</b>ce</mark>?", comment=None),
 Sentence(id=20, line=10, sentence ="S09-Sí, <b>cla</b>ro, hoy en <b>dí</b>a tam<b>bién</b> <mark class='G' title='cacao est complément d&#039;objet indirect donc=&#034;le&#034; et non pas &#034;lo&#034;'>le</mark> <b>e</b>chan a<b>zú</b>car,", comment=None),
 Sentence(id=21, line=11, sentence ="S10-<b>pe</b>ro en la tradi<b>ción</b> in<b>dí</b>gena no se <mark class='G' title='cacao est complément d&#039;objet indirect donc=&#034;le&#034; et non pas &#034;lo&#034;'>le</mark> a<b>ña</b>de <b>le</b>che, <b>so</b>lo <b>a</b>gua.", comment=None),
 Sentence(id=22, line=12, sent

In [66]:
sfdb[2]

Sentence(id=12, line=10, sentence ='S09-Sí, <b>cla</b>ro, hoy en <b>dí</b>a tam<b>bién</b> <mark class=\'G\' title=\'cacao est complément d\\\'objet indirect donc "le" et non pas "lo"\'>le</mark> <b>e</b>chan a<b>zú</b>car,', comment=None)

In [69]:
for index, char in enumerate(sfdb[2].sentence):
    print(f"char : {char}, {ord(char)}, {index}")

char : S, 83, 0
char : 0, 48, 1
char : 9, 57, 2
char : -, 45, 3
char : S, 83, 4
char : í, 237, 5
char : ,, 44, 6
char :  , 32, 7
char : <, 60, 8
char : b, 98, 9
char : >, 62, 10
char : c, 99, 11
char : l, 108, 12
char : a, 97, 13
char : <, 60, 14
char : /, 47, 15
char : b, 98, 16
char : >, 62, 17
char : r, 114, 18
char : o, 111, 19
char : ,, 44, 20
char :  , 32, 21
char : h, 104, 22
char : o, 111, 23
char : y, 121, 24
char :  , 32, 25
char : e, 101, 26
char : n, 110, 27
char :  , 32, 28
char : <, 60, 29
char : b, 98, 30
char : >, 62, 31
char : d, 100, 32
char : í, 237, 33
char : <, 60, 34
char : /, 47, 35
char : b, 98, 36
char : >, 62, 37
char : a, 97, 38
char :  , 32, 39
char : t, 116, 40
char : a, 97, 41
char : m, 109, 42
char : <, 60, 43
char : b, 98, 44
char : >, 62, 45
char : b, 98, 46
char : i, 105, 47
char : é, 233, 48
char : n, 110, 49
char : <, 60, 50
char : /, 47, 51
char : b, 98, 52
char : >, 62, 53
char :  , 32, 54
char : <, 60, 55
char : m, 109, 56
char : a, 97, 57
char : 

In [70]:
print(sfdb[2].sentence)

S09-Sí, <b>cla</b>ro, hoy en <b>dí</b>a tam<b>bién</b> <mark class='G' title='cacao est complément d\'objet indirect donc "le" et non pas "lo"'>le</mark> <b>e</b>chan a<b>zú</b>car,
